In [3]:
using Revise
using ProgressMeter
using ITensors
using RandomMeas

Precompiling RandomMeas
  ✓ RandomMeas
  1 dependency successfully precompiled in 5 seconds. 275 already precompiled.


In [25]:
#Perform RM in the first experiment
N = 6
ξ = siteinds("Qubit", N)
depth = 2
circuit = RandomCircuit(ξ, depth)
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);
#noisemodel1 = (1 => ("depolarizing", (p = 0,)),2 => ("depolarizing", (p = 0.05,)))
ρ1 = apply(circuit,ρ0,apply_dag=true);
#ξ = firstsiteinds(ρ1;plev=0)
#println(ξ)
println("state prepared in the `experiment' 1 ")

state prepared in the `experiment' 1 


In [26]:
#Perform Randomized measurements
nu=200 #Number of random unitaries
NM=1000 #Number of projective measurements per unitary
u = Vector{Vector{ITensor}}()
data1 = zeros(Int8,(nu,NM,N)) #Data storage
@showprogress dt=1 for r in 1:nu
            push!(u,get_rotations(ξ,1)) #generate random unitary
            data1[r,:,:] = get_RandomMeas(ρ1,u[r],NM) #data acquisation in simulated quantum device
end

In [27]:
#Perform RM in the second experiment
# noisemodel2 = (1 => ("amplitude_damping", (γ = 0.1,)),2 => ("amplitude_damping", (γ = 0.1,)))
# ρ2 = runcircuit(circuit;noise = noisemodel2);
# ξ2 = [firstind(ρ2[i],tags="Site",plev=0) for i in 1:N]
# for i in 1:N
#     replaceind!(ρ2[i],ξ2[i],ξ[i])
#     replaceind!(ρ2[i],ξ2[i]',ξ[i]')
# end
p = 0.1*rand(N)
ρ2 = Apply_depo_channel(ρ1,p)
println("state prepared in the `experiment' 2 ")
data2 = zeros(Int8,(nu,NM,N))
@showprogress dt=1  for r in 1:nu
                data2[r,:,:] = get_RandomMeas(ρ2,u[r],NM)
end

state prepared in the `experiment' 2 


In [28]:
#Build histograms for each unitary
P1 = Vector{ITensor}()
P2 = Vector{ITensor}()
for r in 1:nu
                push!(P1,get_Born(data1[r,:,:],ξ))
                push!(P2,get_Born(data2[r,:,:],ξ))
end

In [29]:
#Uncomment if you want to see an animation
# using Plots
# anim = @animate for r in 1:nu
#     y1 =  reshape(array(P1[r]),(2^N))
#     y2 =  reshape(array(P2[r]),(2^N))
#     bar(y1,label="Experiment 1")
#     bar!(y2,label="Experiment 2")
#     xlabel!("bitstring index")
#     ylabel!("Probabilities")
#     title!("Random unitary $(r)")
# end
# gif(anim,fps=0.5)

In [31]:
overlaps = 0
for r in 1:nu
    global overlaps += get_overlap(P1[r],P2[r],ξ,N)/nu
end
purity1s = get_purity_hamming(data1,ξ)
purity2s = get_purity_hamming(data2,ξ)
Fs = overlaps/max(purity1s,purity2s)
println("Estimated overlap ",overlaps)
println("Estimated purity1 ",purity1s)
println("Estimated purity2 ",purity2s)
println("Estimated fidelity ",Fs)

Estimated overlap 0.8184081549999999
Estimated purity1 1.0256800000000004
Estimated purity2 0.6580104804804806
Estimated fidelity 0.7979176302550499


In [32]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("overlap ",overlap)
println("purity1 ",purity1)
println("purity2 ",purity2)
println("fidelity ",F)

overlap 0.802176374003288
purity1 0.9999999999999986
purity2 0.6483053104806866
fidelity 0.8021763740032891
